In [7]:
# infra
job_managers_qty = 2
kafka_qty = 8
kafka_partitions=32
kafka_replicas=8
kafka_node_selector="jobmanager"
# eval
events_num = 600000000
source_parallelism = 32
memory_range = [8192,16384,32768,65536] 
#memory_range = [16384] 
task_slots_limits = [source_parallelism + 11, source_parallelism + 18, source_parallelism + 25, source_parallelism + 32]
#task_slots_limits = [source_parallelism + 32]
results_path = "verification-results/"
monitoring_step = "5s"
reset_kafka_data = True
limit_backpressure_source = 0
notebooks = ["/xp_intro_q5_kafka_custom_ratelimit"]
cpu = 16
task_slots_per_task_manager = 16
task_managers_qty = 4
run = 0
warmup = 120
nb_runs_throughput = 3
nb_runs_parallelism = 1
kind=False
g5k=False
proxmox = False
xp_name="query"

dichotomic_mst_tuning = {
    "initial_rate": 10**8,
    "slide_window": 75,
    "size_window": 60,
    "observation_size": 30,
    "timeout": 600,
    "mean_threshold": 0.01,
    "higher_bound_ratio": 2,
    "cooldown_throughput": 200,
    #"warmup": 120 # usage of higher level parameter
    #"nb_sources": usage of of higher level parameter
}

datagen_configuration = {
    "parallelism": 64,
    "cpu": 4,
    "memory" : 4096,
    "task_managers_qty": 4,
    "task_slots": 2
}

In [2]:
#papermill_description=Initialize libraries
import sys
import os
from pathlib import Path 
from time import sleep
import logging 
%run ../common/scripts.ipynb
if "../streambed" not in sys.path:
    sys.path.append("../streambed")
import streambed
streambed.setup_logging(default_path="./logging.yaml", default_level=logging.WARN)
logger = logging.getLogger('streambed')
if kind:
    streambed.init_kind()
if g5k:
    streambed.init_g5k()
if proxmox:
    streambed.init_remote()
logging.info("Execute notebook")

/home/grosinosky/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/grosinosky/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


In [9]:
#papermill_description=Reset_Kafka
if reset_kafka_data:
    #papermill_description=Initialize libraries
    init_label_nodes(jobmanagers_qty=job_managers_qty, kafka_qty=kafka_qty)

    address = "127-0-0-1"
    port=30081

    !kubectl delete -n kafka kafka/my-cluster
    sleep(10)
    #run_command("kubectl create -f ./kafka.yaml", shell=False)
    streambed.deploy_kafka(kafka_partitions, kafka_replicas, node_selector=kafka_node_selector, antiaffinity=g5k)
    run_command("kubectl wait kafka/my-cluster --for=condition=Ready --timeout=300s -n kafka", shell=False)
    # Kafka UI
    run_command("helm install -f values-kowl.yaml -n kafka kowl cloudhut/kowl", shell=False)
    #(ip_url, dns_url) = get_service_public_address("kafka", "manager", "kowl", 80)
    (manager_node, jobmanager_node, taskmanager_node) = get_label_nodes()
    print("Kafka Kowl: {} - Ingress: {}".format(manager_node, "http://kowl.{}.sslip.io:{}".format(address,port)))    
    !kubectl apply -f ./kafka-bridge.yaml
    !kubectl apply -f ./kafka-bridge-service.yaml

kafka.kafka.strimzi.io "my-cluster" deleted
kubectl wait kafka/my-cluster --for=condition=Ready --timeout=300s -n kafka
kubectl wait kafka/my-cluster --for=condition=Ready --timeout=300s -n kafka
Log 0 - 2023-04-13 00:27:04 : kafka.kafka.strimzi.io/my-cluster condition met
helm install -f values-kowl.yaml -n kafka kowl cloudhut/kowl
helm install -f values-kowl.yaml -n kafka kowl cloudhut/kowl
Log 0 - 2023-04-13 00:27:04 : WARNING: Kubernetes configuration file is group-readable. This is insecure. Location: /home/grosinosky/.kube/config
Log 0 - 2023-04-13 00:27:04 : WARNING: Kubernetes configuration file is world-readable. This is insecure. Location: /home/grosinosky/.kube/config
Log 0 - 2023-04-13 00:27:04 : Error: cannot re-use a name that is still in use
gros-1.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-1.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.i

You need to set Flink interpreter in per note/isolated mode to allow parallel Flink notebooks (datagen & tested query) in Zeppelin: (Zeppelin UI : Interpreters / Flink / "The interpreter will be instantiated *Per Note* in *Isolated* process)
```bash
export ZEPPELIN_POD=`kubectl get pods -n manager -o=name | grep zeppelin | sed "s/^.\{4\}//"` 
export MANAGER_NODE=`kubectl get node --show-labels |grep tier=manager | awk '{print $1}'`

kubectl -n manager exec $ZEPPELIN_POD -- wget -P /tmp https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-connector-kafka_2.12/1.14.2/flink-sql-connector-kafka_2.12-1.14.2.jar
# nexmark and the rate limit connector should be both in the tmp directory
kubectl cp -n manager ./tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar $ZEPPELIN_POD:/tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar
kubectl cp -n manager ./tmp/nexmark-flink-0.2-SNAPSHOT.jar $ZEPPELIN_POD:/tmp/nexmark-flink-0.2-SNAPSHOT.jar
# upload zeppelin notebooks (replace gros-108 by your manager node address/ip)
for FILE in ./xp/streambed-nexmark/zeppelin/*.zpln; do echo $FILE; curl ${MANAGER_NODE}:30088/api/notebook/import -d @$FILE; done

# remove from 
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-connector-kafka_2.12-1.14.2.jar
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-sql-connector-kafka_2.12-1.14.2.jar
```

In [4]:
#papermill_description=Initialize_data
if reset_kafka_data:
    streambed.deploy("nexmark", streambed.flink_configuration(cpu=cpu,memory=max(memory_range),task_managers=task_managers_qty, task_slots=task_slots_per_task_manager, evenly_spread="true", custom_memory=None))
    if streambed.common.dynamic_flink_url:
        (_, base_url) = get_service_public_address("default", "jobmanager", "nexmark-flink-jobmanager", 8081)
        streambed.common.flink_base_url = base_url
    streambed.wait_for_task_managers(task_managers_qty=task_managers_qty, base_url=streambed.common.flink_base_url)

    streambed.delete_kafka_topic("nexmark")
    sleep(10)
    streambed.launch_job_async(notebook="/xp_intro_init_kafka", params_datagen={
        "params":{
            "TOPIC" : "nexmark",
            "BOOTSTRAP_SERVERS": "my-cluster-kafka-bootstrap.kafka:9092",
            "TPS": "1000000000",
            "EVENTS_NUM": str(events_num),
            "PERSON_PROPORTION" : "1",
            "AUCTION_PROPORTION": "3",
            "BID_PROPORTION": "46",
            "NEXMARK_TABLE": "kafka"
        }}, params_query = {}, timeout=1200)


(None,
 [{'title': 'Flink packages configurations',
   'text': '%flink.conf\n\n# nexmark should have be copied in tmp before with this kind of command: kubectl -n manager cp nexmark-flink-0.2-SNAPSHOT.jar $(kubectl get pods -n manager -l app.kubernetes.io/name=zeppelin-server  -o jsonpath="{.items[0].metadata.name}"):/tmp\n\nflink.execution.mode remote\nflink.execution.remote.host nexmark-flink-jobmanager.default\nflink.execution.remote.port 8081\n\n#table.exec.mini-batch.enabled: true\n#table.exec.mini-batch.allow-latency: 2s\n#table.exec.mini-batch.size: 50000\ntable.optimizer.distinct-agg.split.enabled: true\ntable.optimizer.simplify-operator-name-enabled: false\n\n# You need to run this paragraph first before running any flink code.\n\n#flink.execution.jars /tmp/nexmark-flink-0.2-SNAPSHOT.jar,/opt/flink/lib/flink-json-1.14.2.jar,/opt/flink/lib/flink-sql-connector-kafka_2.12-1.14.2.jar,/tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar\n\nflink.execution.jars /tmp/nexmark-flin

In [8]:
#papermill_description=Streambed_estimation
initial_parallelism = 1

list_xp=[]
for notebook in notebooks:
    for memory in memory_range:
        for task_slots_limit in task_slots_limits:
            list_xp.append(
                {"cpu": cpu, 
                "memory": memory, 
                "run": run, 
                "task_slots_per_task_manager": task_slots_per_task_manager, 
                "task_managers_qty": task_managers_qty,
                "source_parallelism": source_parallelism, 
                "parallelism": initial_parallelism, 
                "evenly_spread": "true", 
                "warmup": warmup,
                "custom_memory": None, 
                "dichotomic_mst_tuning": dichotomic_mst_tuning,
                "task_slots_limit": task_slots_limit,
                "throughputs": [10**8],
                "timeout": 600,
                "limit_backpressure_source": limit_backpressure_source,
                "monitoring_step": monitoring_step,                
                "nb_runs_throughput": nb_runs_throughput,
                "nb_runs_parallelism": nb_runs_parallelism,
                "notebook": notebook})
import datetime            
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d%H%M%S")     
path= "{}/{}-{}".format(results_path, xp_name, now_str)
results_estimation = streambed.loop_estimation(list_xp, results_path=path)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

IndexError: list index out of range